In [56]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.io as sio
#from Lorenz96_RK4 import Lorenz96
from L96_Model import L96
import sys, os


In [76]:
# Data, Data Assimilation Hyperparameters
D = 20
Dobs = 12
dims = set(np.arange(20))
#dim_obs = [1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 18, 19]
dim_obs = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16, 17, 18]
dim_unobs = list(dims - set(dim_obs))
M = 200

# Annealing Hyperparameters
Rm = 1
Rf0 = 1e6
alpha = 1.3
betamax = 1
# Temperature
Te = np.exp(1e-1*np.arange(niter))


# Hamiltonian Monte Carlo Hyperparameters
niter = int(1e3)
epsilon = 1e-3*np.ones((1,niter))
L = 150
mass = [1e3, 1e-2, 1e0]

# Post-processing
plot_Action_vs_beta = False
savedata = False

In [58]:
# Load the data set
gen_nu = "8.17"
gen_noise = "sig0.4"
gen_dt = "0.001"
gen_delta_t = "0.025"
gen_integrator = "RK4"

# Specify the data path
datapath = ("./L96_D%s_nu%s_%s_dt%s_deltat%s_%s.mat" % 
    (str(D), gen_nu, gen_noise, gen_dt, gen_delta_t, gen_integrator))

if datapath[-3:] == "mat":
    datadict = sio.loadmat(datapath)
    data = datadict["Data"]
    dt = datadict["delta_t"]
elif datapath[-3:] == "npy":
    data = np.load(datapath)
else:
    raise IOError ("Does not recognize data file extension\n datapath = %s" % datapath)
    sys.exit()

In [85]:
# Prepare the data
Y = data[dim_obs,:M]

#Initialize the state variables
np.random.seed(12345)
X_init = np.zeros((D,M))
X_init[:,0] = 20*np.random.random(size=(D))
X_init[dim_obs, :] = Y

for i in range(1, M): # should be M -1??
    X_init[:,i] = X_init[:,i-1] + dt * 

In [89]:

print(X_init[:,0])

[-1.48508531  1.11390342 -1.13540662  4.09120557  1.94734112 11.91089406
 -1.91878251 13.06354194 -0.6047727  13.07139742 -0.98195435 19.22613472
  4.32478077  2.12888753  7.00115957 13.12822366  2.65478063  1.86368711
  2.13376556 14.47370694]


In [ ]:
def generate_data():
    """
        This function is optional, in case you do not want to use previous data
    """
    # Set the hyperparamters for generating the data
    # stepsize = dt
    stepsize = 0.01

    # How many time points
    steps = 1000

    # Initial point
    np.random.seed(123456)
    D = 20
    y0 = np.random.random(D)*3
    t0 = 0
    F = 8

    sample = Lorenz96(y0, t0, D, F)
    data = sample.generate(stepsize=stepsize, steps=steps, addNoise=True, noise_level=0.04)
    return data